## Timeseries на PySpark

In [2]:
from datetime import datetime

from pyspark import SparkContext, SQLContext
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, TimestampType, DoubleType, StringType

from sparkts.datetimeindex import uniform, BusinessDayFrequency

# timeseries библиотека
from sparkts.timeseriesrdd import time_series_rdd_from_observations

def lineToRow(line):
    # функция преобразования
    (year, month, day, symbol, volume, price) = line.split("\t")
    # Совместимый для всех Python
    dt = datetime(int(year), int(month), int(day))
    return (dt, symbol, float(price))

def loadObservations(sparkContext, sqlContext, path):
    textFile = sparkContext.textFile(path)
    rowRdd = textFile.map(lineToRow)
    schema = StructType([
        StructField('timestamp', TimestampType(), nullable=True),
        StructField('symbol', StringType(), nullable=True),
        StructField('price', DoubleType(), nullable=True),
    ])
    return sqlContext.createDataFrame(rowRdd, schema);

In [4]:
# оди из вариантов контекст + данные = df + init
tickerObs = loadObservations(sc, sqlContext, "ticker.tsv")

In [ ]:
tickerObs.show(2, truncate=False)

In [92]:
# Переводим время в форма
freq = BusinessDayFrequency(1, 1, sc)
dtIndex = uniform(start='2015-08-03T00:00+01:00', end='2015-09-22T00:00+01:00', freq=freq, sc=sc)

In [ ]:
dtIndex

In [94]:
# формируем DF на подобии Pandas DF (как особенности в данных для TS)
tickerTsrdd = time_series_rdd_from_observations(dtIndex, tickerObs, "timestamp", "symbol", "price")

In [ ]:
tickerTsrdd.take(20)

In [ ]:
tickerTsrdd.cache()

# кол-во элементов
print(tickerTsrdd.count())

# заменяем пропущенные значения (интерполиряция). Где такой способ "на не помогает"?
filled = tickerTsrdd.fill("linear")

# рейт возвращения
returnRates = filled.return_rates()

# Durbin-Watson (нужно описать самому, подобее функции TimeSeriesStatisticalTests.scala)
# вычисления на основе остатков
def dwtest(residuals):
    residsSum = residuals[0] * residuals[0]
    diffsSum = 0.0
    i = 1
    while i < len(residuals):
        residsSum += residuals[i] * residuals[i]
        diff = residuals[i] - residuals[i - 1]
        diffsSum += diff * diff
        i += 1
    return diffsSum / residsSum

# Применим функцию
dwStats = returnRates.map_series(lambda row: (row[0], [dwtest(row[1])])).map(lambda x: (x[1], x[0]))

print(dwStats.min())
print(dwStats.max())

In [ ]:
returnRates.take(2)

In [ ]:
tickerTsrdd.take(2)

### ARIMA

In [102]:
from sparkts.models import ARIMA

from pyspark.mllib.linalg import Vectors

In [131]:
ts = Vectors.dense(returnRates.collect()[0][1])

In [135]:
m = ARIMA.fit_model(1,0,1,ts, sc=sc) 

In [136]:
# коэф. модели
m.coefficients

[-0.0012474547399206936, 0.2666755318221923, 0.0961795619837402]

In [137]:
# предикт на 20 шагов
forecast = m.forecast(ts, 20)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(forecast.toArray(), label="Forecast")
plt.plot(ts.toArray(), label="Timeseries")
plt.legend(loc="best")

In [ ]:
dwStats.collect()

### Применение Dickey Fueller теста

In [ ]:
import numpy as np
import statsmodels.tsa.stattools as ts

In [ ]:
# функция применяется к RDD
returnRates.map(lambda x: (x[0], ts.adfuller(x[1],1))).take(2)

## Durbin Watson

In [186]:
import statsmodels.api as sm

# применение метода из statsodels
returnRates.map(lambda x: (x[0], sm.stats.durbin_watson(x[1]))).take(2)

[(u'AAL', 1.2651505598339603), (u'AAPL', 1.9645069225856866)]

### Применение модели из Statsmodels

In [198]:
# rdd позволяет применять методы и модели
arima_101 = returnRates.map(lambda x: (x[0], sm.tsa.ARIMA(x[1], (1,0,1)).fit()))

In [199]:
# обзор параметров (коэф)
arima_101.map(lambda res_arima:(res_arima[0], res_arima[1].params) ).take(1)

[(u'AAL', array([-0.00037234,  0.22196278,  0.12971846]))]